In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import nn
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import onnxruntime
from sklearn.model_selection import KFold
from model import Model
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
device = torch.device("cuda")

In [2]:
model = Model()
model.load_state_dict(torch.load('Save_Model/model_parameter-2023-5-31-1.pkl'))

<All keys matched successfully>

In [3]:
dynamic_axes_0 = { 
    'input' : {0: 'batch'}, 
    'output' : {0: 'batch'} 
} 


x = torch.randn(1,1,1920)
x.cpu()
model.cpu()
model.eval()
torch_out = torch.onnx.export(model,x,"Save_Model/onnx_model.onnx",input_names=['input'],output_names=['output'],opset_version=11,dynamic_axes=dynamic_axes_0)

/home/wucheng/.virtualenvs/tf/lib/python3.7/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)
/home/wucheng/.virtualenvs/tf/lib/python3.7/site-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [4]:
import onnx
try:
    # 当我们的模型不可用时，将会报出异常
    onnx.checker.check_model("Save_Model/onnx_model.onnx")
except onnx.checker.ValidationError as e:
    print("The model is invalid: %s"%e)
else:
    print("The model is valid!")

The model is valid!


In [5]:
class ONNXModel():
    def __init__(self, onnx_path):
        """
        :param onnx_path:
        """
        self.onnx_session = onnxruntime.InferenceSession(onnx_path)
        self.input_name = self.get_input_name(self.onnx_session)
        self.output_name = self.get_output_name(self.onnx_session)
        print("input_name:{}".format(self.input_name))
        print("output_name:{}".format(self.output_name))

    def get_output_name(self, onnx_session):
    
        output_name = []
        for node in onnx_session.get_outputs():
            output_name.append(node.name)
        return output_name

    def get_input_name(self, onnx_session):
    
        input_name = []
        for node in onnx_session.get_inputs():
            input_name.append(node.name)
        return input_name

    def get_input_feed(self, input_name, image_numpy):

        input_feed = {}
        for name in input_name:
            input_feed[name] = image_numpy
        return input_feed

    def forward(self, image_numpy):
 
        # scores, boxes = self.onnx_session.run(None, {self.input_name: image_numpy})
        # scores, boxes = self.onnx_session.run(self.output_name, input_feed={self.input_name: iimage_numpy})
        input_feed = self.get_input_feed(self.input_name, image_numpy)
        scores, boxes = self.onnx_session.run(self.output_name, input_feed=input_feed)
        return scores, boxes
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [6]:
test_x = np.load('data/X_test.npy')
test_y = np.load('data/y_seg_test.npy')
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))
test_y = test_y.reshape((test_y.shape[0], test_y.shape[1]))
test_x = torch.FloatTensor(test_x)
test_y  = torch.FloatTensor(test_y)

In [7]:
onnx_model_path = "Save_Model/onnx_model.onnx"
sess = onnxruntime.InferenceSession(onnx_model_path,providers=['CPUExecutionProvider']) # 'CPUExecutionProvider'
input_name = sess.get_inputs()[0].name
output_name = [output.name for output in sess.get_outputs()]

# 推理
import datetime
start = datetime.datetime.now()
outputs = sess.run(output_name, {input_name:to_numpy(test_x[1:2])})
end = datetime.datetime.now()
print(end-start)
print("onnx weights", outputs)
print("onnx prediction", outputs[0])